### 1.Use the Notebook to build the code to scrape the following Wikipedia page: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [1]:
!conda install bs4 --yes
from bs4 import BeautifulSoup
from urllib.request import urlopen
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd

!conda install -c conda-forge folium=0.5.0 --yes
import folium
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
print('import done')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - bs4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.9.0       |           py36_0         167 KB
    bs4-4.9.0                  |                0           9 KB
    ca-certificates-2020.1.1   |                0         125 KB
    certifi-2020.4.5.1         |           py36_0         155 KB
    openssl-1.1.1g             |       h7b6447c_0         2.5 MB
    soupsieve-2.0              |             py_0          33 KB
    ------------------------------------------------------------
                                           Total:         3.0 MB

The following NEW packages will be INSTALLED:

  beautifulsoup4     pkgs/main/linux-64::beautifulsoup4-4.9.0-py36_0
  bs4                pkgs/main/noarch::bs4-4.9.0-0
  soupsieve          pkg

Obtain table data from Wikipedia page using the BeautifulSoup package.

In [2]:
import urllib.request
with urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M') as f:
    toronto_N = BeautifulSoup(f)

    
match = toronto_N.find('table', class_='wikitable sortable')

import pandas as pd
import numpy as np
rows = match.find_all('tr')


dt_torontotext=[]

for r in range(len(rows)):
    dt_toronto = rows[r].find_all('td')
    temp=[]
    for val in dt_toronto:
        temp.append(val.text[:-1])
    dt_torontotext.append(temp)    


 Abstract three columns from postal codes table: PostalCode, Borough, and Neighborhood

In [3]:
col =  match.find_all('th')
column = [value.text[:-1] for value in col]
print(column) 


['Postal code', 'Borough', 'Neighborhood']


Transform the data into a pandas dataframe.

In [4]:
df = pd.DataFrame(dt_torontotext, columns = column)
df = df.drop(0)
df.head()

,Postal code,Borough,Neighborhood
1,M1A,Not assigned,
2,M2A,Not assigned,
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Regent Park / Harbourfront


Process the cells that have an assigned borough. Drop cells with a borough that is Not assigned.
Use the .shape method to print the number of rows of your dataframe.

In [5]:
df_new = df.loc[df['Borough']!='Not assigned']
df_n = df_new.reset_index(drop=True)
print(df_n.head())
print(df_n.shape)

  Postal code           Borough                                  Neighborhood
0         M3A        North York                                     Parkwoods
1         M4A        North York                              Victoria Village
2         M5A  Downtown Toronto                    Regent Park / Harbourfront
3         M6A        North York             Lawrence Manor / Lawrence Heights
4         M7A  Downtown Toronto  Queen's Park / Ontario Provincial Government
(103, 3)


Check the existance of Not assigned data in neighborhood column. 

In [6]:

for i in df['Neighborhood']:
    if i=='Not assigned':
        print(i)   
print('Neighborhood all assigned')     
 

Neighborhood all assigned


### 2. Get the latitude and the longitude coordinates of each neighborhood.

Find geographical coordinates of each postal code from http://cocl.us/Geospatial_data

In [7]:
df = pd.read_csv('http://cocl.us/Geospatial_data')
 
df.rename(columns={'Postal Code':'Postal code'}, inplace=True)
print(df.head())

  Postal code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476


In [8]:
pd_cor = pd.merge(left=df_n, right=df, how='left', left_on='Postal code', right_on='Postal code')
print(pd_cor)
print(pd_cor.shape)


    Postal code           Borough  \
0           M3A        North York   
1           M4A        North York   
2           M5A  Downtown Toronto   
3           M6A        North York   
4           M7A  Downtown Toronto   
..          ...               ...   
98          M8X         Etobicoke   
99          M4Y  Downtown Toronto   
100         M7Y      East Toronto   
101         M8Y         Etobicoke   
102         M8Z         Etobicoke   

                                          Neighborhood   Latitude  Longitude  
0                                            Parkwoods  43.753259 -79.329656  
1                                     Victoria Village  43.725882 -79.315572  
2                           Regent Park / Harbourfront  43.654260 -79.360636  
3                    Lawrence Manor / Lawrence Heights  43.718518 -79.464763  
4         Queen's Park / Ontario Provincial Government  43.662301 -79.389494  
..                                                 ...        ...        ...  
98